In [1]:
# get working directory
import tkinter as tk
from tkinter import filedialog
import os
import sys
import numpy as np

SCRIPT_DIR = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
sys.path.append(os.path.dirname(SCRIPT_DIR))

import onnx
from monai.networks.nets import BasicUNet, UNet
import torch


/Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: /Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/torchvision/image.so
  Expected in: /Users/jasonfung/miniforge3/envs/ml_env/lib/python3.9/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
model_state_path = '/Users/jasonfung/Documents/Label_Seg_Program/models/-s +d -f_ResUNet_90.pth'

input_chnl = 1
output_chnl = 2

unet_model = UNet(spatial_dims=3, 
             in_channels = input_chnl,
             out_channels = output_chnl,
             channels = (32, 64, 128, 256, 512),
             strides=(2, 2, 2, 2),
             num_res_units=2,
             norm = "batch",
             #dropout = dropout,
             )

map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None
unet_model.load_state_dict(torch.load(model_state_path, map_location=map_location))

unet_model.eval()

UNet(
  (model): Sequential(
    (0): ResidualUnit(
      (conv): Sequential(
        (unit0): Convolution(
          (conv): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
          (adn): ADN(
            (N): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
        (unit1): Convolution(
          (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (adn): ADN(
            (N): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
      )
      (residual): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): ResidualUnit(
          (conv): Sequential(


In [3]:
batch_size = 64
input_size = (batch_size,1,16,64,64)
x = torch.randn(input_size)
x = x/x.max()

In [4]:
x.max()

tensor(1.)

In [5]:
torch_out = unet_model(x)

In [6]:
torch_out.shape

torch.Size([64, 2, 16, 64, 64])

In [7]:
torch.onnx.export(unet_model,
                  x,
                  "Neuron.onnx",
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes = {'input': {0:"batch_size"},
                                  'output': {0:"batch_size"}})

In [10]:
import onnx

onnx_model = onnx.load("/Users/jasonfung/Documents/Label_Seg_Program/models/Neuron.onnx")
onnx.checker.check_model(onnx_model)

In [11]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("/Users/jasonfung/Documents/Label_Seg_Program/models/Neuron.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

AssertionError: 
Not equal to tolerance rtol=0.001, atol=1e-05

Mismatched elements: 8 / 8388608 (9.54e-05%)
Max absolute difference: 7.6293945e-05
Max relative difference: 1.4234234
 x: array([[[[[  21.25904 ,   30.27383 ,   36.432766, ...,   29.909758,
             24.33224 ,   14.131004],
          [  29.751183,   43.541187,   50.496635, ...,   36.70154 ,...
 y: array([[[[[  21.259047,   30.273836,   36.432766, ...,   29.909761,
             24.332245,   14.131004],
          [  29.751186,   43.54119 ,   50.49664 , ...,   36.701546,...

In [15]:
ort_outs[0][0]

array([[[[ 21.259047,  30.273836,  36.432766, ...,  29.909761,
           24.332245,  14.131004],
         [ 29.751186,  43.54119 ,  50.49664 , ...,  36.701546,
           30.777832,  18.427338],
         [ 26.143398,  39.116367,  49.251747, ...,  20.279957,
           18.128325,  12.043082],
         ...,
         [ 49.118145,  52.666138,  32.44316 , ...,  38.30459 ,
           47.52028 ,  36.750076],
         [ 32.66972 ,  33.950573,  26.50837 , ...,  36.585396,
           42.601757,  29.04014 ],
         [ 17.995262,  16.43135 ,  11.18037 , ...,  29.823708,
           29.70953 ,  18.975857]],

        [[ 36.109478,  45.826492,  39.97216 , ...,  30.76885 ,
           28.752956,  20.248852],
         [ 52.65888 ,  63.989555,  50.93632 , ...,  31.833021,
           31.330273,  21.52916 ],
         [ 47.887196,  59.550697,  47.001938, ...,  16.104273,
           15.537632,  10.747179],
         ...,
         [ 44.607994,  53.612484,  51.917465, ...,  38.553886,
           63.563675,  53

In [16]:
torch_out[0]

tensor([[[[ 21.2590,  30.2738,  36.4328,  ...,  29.9098,  24.3322,  14.1310],
          [ 29.7512,  43.5412,  50.4966,  ...,  36.7015,  30.7778,  18.4273],
          [ 26.1434,  39.1164,  49.2518,  ...,  20.2800,  18.1283,  12.0431],
          ...,
          [ 49.1181,  52.6661,  32.4432,  ...,  38.3046,  47.5203,  36.7501],
          [ 32.6697,  33.9506,  26.5084,  ...,  36.5854,  42.6018,  29.0401],
          [ 17.9953,  16.4314,  11.1804,  ...,  29.8237,  29.7095,  18.9759]],

         [[ 36.1095,  45.8265,  39.9722,  ...,  30.7688,  28.7529,  20.2488],
          [ 52.6589,  63.9895,  50.9363,  ...,  31.8330,  31.3303,  21.5292],
          [ 47.8872,  59.5507,  47.0019,  ...,  16.1043,  15.5376,  10.7472],
          ...,
          [ 44.6080,  53.6125,  51.9175,  ...,  38.5539,  63.5637,  53.8403],
          [ 36.1950,  50.4958,  55.1523,  ...,  42.5150,  55.9018,  42.5217],
          [ 27.3911,  33.8150,  38.2481,  ...,  36.3848,  38.2925,  23.0091]],

         [[ 50.7190,  59.9645,

In [14]:
torch_out[0].shape

torch.Size([2, 16, 64, 64])